In [16]:
%load_ext autoreload
%autoreload 2
import numpy as np
from helper_functions import *
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import KFold
from sklearn.svm import SVC


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
get_score_diff(1422837176)

7

In [5]:
path = "tweet_data/"

files = ["tweets_#gohawks.txt", "tweets_#gopatriots.txt", 
        "tweets_#nfl.txt", "tweets_#patriots.txt", 
        "tweets_#sb49.txt", "tweets_#superbowl.txt"]
tweets_gopatriots = load_tweets("tweet_data/tweets_#gopatriots.txt")
tweets_gohawks = load_tweets("tweet_data/tweets_#gohawks.txt")
tweets_superbowl = load_tweets("tweet_data/tweets_#superbowl.txt")

In [7]:
tweets = tweets_gopatriots + tweets_gohawks + tweets_superbowl
print(len(tweets))
print(len(tweets_gopatriots))
print(len(tweets_gohawks))
print(len(tweets_superbowl))

29151
299
6108
22744


In [20]:
# print(tweets_gopatriots[0])
# print(tweets_gohawks[0])
# print(tweets_superbowl[0])

labels = list()
features = list()
for tweet in tweets:
    labels.append(get_tweet_polarity(tweet['tweet']['text']))
    features.append(extract_features(tweet))

In [30]:
print(labels[0])
print(features[0])


Positive
{'timestamp': 1422888857.0, 'Massachusetts': 1, 'Washington': 0, 'Retweet Count': 0, 'Score Diff': 4}


In [31]:
vec = DictVectorizer(sparse=False)
train_features = vec.fit_transform(features)
print(train_features)


[[  1.00000000e+00   0.00000000e+00   4.00000000e+00   0.00000000e+00
    1.42288886e+09]
 [  1.00000000e+00   0.00000000e+00   7.00000000e+00   0.00000000e+00
    1.42283625e+09]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    1.42283342e+09]
 ..., 
 [  0.00000000e+00   0.00000000e+00   4.00000000e+00   1.00000000e+00
    1.42329564e+09]
 [  1.00000000e+00   0.00000000e+00   4.00000000e+00   0.00000000e+00
    1.42331123e+09]
 [  1.00000000e+00   0.00000000e+00   4.00000000e+00   0.00000000e+00
    1.42332326e+09]]


In [ ]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
for train_index, test_index in kf.split(features_array):
    X_train, X_test = train_features[train_index], train_features[test_index]
    y_train, y_test = train_labels[train_index], train_labels[test_index]
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    confusionmatrix = confusion_matrix(y_test, y_pred)
    print('Confusion Matrix:\n', confusionmatrix)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('Micro F1 Score:', f1_score(y_test, y_pred, average='micro'))
    print('Macro F1 Score:', f1_score(y_test, y_pred, average='macro'))
    print('Weighted F1 Score:', f1_score(y_test, y_pred, average='weighted'))
    print('\n')

